<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9479db54c07088e030f873d915219c7826a9973a5317b791b21ce4b9e655aa65
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 09:55:40
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -30.44 K (-0.21%)
Current PnL: -22.96 L (-15.05%)
CY Booked + Current PnL: -9.07 L (-5.94%)
-------------------
Total profit:  1.58 L
Total loss:  -24.55 L
-------------------
Total Booked + Current PnL: 17.91 L (14.26%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.72%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.37 L (61.87%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,0.96,-13.41,15.59,0.09,13605.0,-13511.0,87266.0,102.56,53.0,M-SC,4.30,253.0,-0.99,0.61,14.65,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-0.16,-9.13,10.07,0.02,15973.0,-15944.0,158615.0,141.63,71.0,M-SC,5.44,234.0,-1.00,1.11,37.97,OX40N,NTT,PAINTS
83,VOLTAS,1530.0,-0.36,8.80,10.01,19.69,20883.0,16878.0,208620.0,-1.77,53.0,X-MC,3.56,78.0,0.81,1.46,16.20,XY25,NTT,AC
43,ITC,452.0,-0.29,-0.64,11.15,10.44,22172.0,-1286.0,198852.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.89,X40,NTT,FMCG
53,NESTLEIND,1377.0,-0.20,12.68,7.74,21.40,23148.0,33649.0,299075.0,4.55,61.0,X-LC,7.93,12.0,1.45,2.09,20.16,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1922.01,1.37,5.79,17.91,24.73,45836.0,14002.0,255926.0,10.64,64.0,X-LC,4.64,7.0,0.31,1.79,21.90,X40,ATH,IT
85,WIPRO,420.00,1.21,1.50,69.72,72.27,119908.0,2537.0,171985.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.20,7.76,XR,NTT,IT
70,SURYODAY,216.00,1.17,-18.72,51.37,23.03,74771.0,-33517.0,145554.0,57.89,50.0,H-SC,11.17,167.0,-0.45,1.02,44.42,XR,NTT,BANKS
14,CAMPUS,393.00,1.14,-11.52,50.63,33.28,75161.0,-19323.0,148452.0,-28.70,36.0,M-SC,2.96,222.0,-0.26,1.04,15.06,XY24,NTT,FOOTWEAR
50,LTIM,7201.88,1.07,9.50,20.24,31.66,54552.0,23377.0,269528.0,5.43,64.0,H-LC,12.54,43.0,0.43,1.89,49.55,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RAJOOENG,143.10,-3.34,-29.98,99.39,39.61,71332.0,-30730.0,71770.0,-52.57,20.0,H-SC,19.28,135.0,-0.43,0.50,0.00,AR,ATH,MISC
0,5PAISA,593.00,-2.95,-36.65,78.21,12.90,108261.0,-80072.0,138424.0,104.33,55.0,H-SC,14.61,161.0,-0.74,0.97,15.10,OX40N,NTT,FINANCE
81,VALIANTORG,838.00,-2.07,-45.33,218.57,74.15,195482.0,-74168.0,89437.0,-260.95,26.0,H-SC,14.53,149.0,-0.38,0.63,14.64,XR,NTT,CHEMICALS
48,KPIGREEN,730.66,-2.02,-11.45,65.95,46.95,73175.0,-14341.0,110956.0,-18.10,27.0,H-SC,6.27,154.0,-0.20,0.78,39.00,MH,ATH,POWER
27,GLAXO,3466.20,-1.87,-1.60,41.41,39.15,81203.0,-3188.0,196096.0,-20.92,29.0,X-MC,9.05,60.0,-0.04,1.37,26.94,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.99,-0.36,107.47,106.72,132071.0,-445.0,122891.0,-50.63,33.0,H-SC,4.08,139.0,-0.0,0.86,22.93,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.75,-4.70,118.16,107.91,166261.0,-6936.0,140708.0,-22.58,34.0,M-SC,10.34,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.16,-9.13,10.07,0.02,15973.0,-15944.0,158615.0,141.63,71.0,M-SC,5.44,234.0,-1.00,1.11,37.97,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.96,-13.41,15.59,0.09,13605.0,-13511.0,87266.0,102.56,53.0,M-SC,4.30,253.0,-0.99,0.61,14.65,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.28,-22.52,46.46,13.47,97067.0,-60741.0,208926.0,-68.12,27.0,H-SC,1.52,158.0,-0.63,1.46,6.23,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.93,-22.61,70.60,32.03,72033.0,-29805.0,102030.0,-16.20,50.0,H-SC,6.61,148.0,-0.41,0.71,14.28,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.21,1.5,69.72,72.27,119908.0,2537.0,171985.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.2,7.76,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.21,1.50,69.72,72.27,119908.0,2537.0,171985.0,-12.89,57.0,M-LC,4.61,99.0,0.02,1.20,7.76,XR,NTT,IT
38,INDIAMART,4810.62,-0.99,-0.36,107.47,106.72,132071.0,-445.0,122891.0,-50.63,33.0,H-SC,4.08,139.0,-0.00,0.86,22.93,AR,ATH,MISC
25,FINCABLES,1641.55,-0.75,-4.70,118.16,107.91,166261.0,-6936.0,140708.0,-22.58,34.0,M-SC,10.34,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.30,-2.24,38.74,35.63,79370.0,-4696.0,204878.0,-15.85,34.0,H-MC,4.52,119.0,-0.06,1.43,14.06,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-1.25,-6.60,94.77,81.92,101618.0,-7572.0,107226.0,-45.33,30.0,M-SC,5.26,236.0,-0.07,0.75,26.94,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.24,-35.62,110.02,35.20,91134.0,-45836.0,82834.0,8.59,21.0,X-SC,12.65,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,-1.78,-10.68,29.01,15.23,51911.0,-21404.0,178943.0,-23.63,22.0,X-MC,6.53,66.0,-0.41,1.25,18.12,X40N,NTT,REALTY
76,TMPV,600.00,-1.20,-15.19,67.64,42.18,158564.0,-41986.0,234424.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
34,HONAUT,58357.33,0.60,-15.66,69.94,43.32,96070.0,-25512.0,137360.0,-27.57,27.0,X-SC,2.02,90.0,-0.27,0.96,5.47,X40N,ATH,ELECTRICAL
60,RELAXO,1176.00,-0.89,-47.57,190.51,52.31,144982.0,-69058.0,76102.0,-44.15,28.0,X-SC,3.79,91.0,-0.48,0.53,1.87,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.20,-15.19,67.64,42.18,158564.0,-41986.0,234424.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.29,-0.64,11.15,10.44,22172.0,-1286.0,198852.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.89,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.29,1.57,28.94,30.97,51953.0,2783.0,179521.0,-22.29,50.0,X-MC,1.54,75.0,0.05,1.26,14.33,X40,ATH,INSURANCE
3,ACC,3906.0,-0.48,-23.45,114.45,64.15,208459.0,-55811.0,182140.0,-55.44,41.0,X-SC,1.90,82.0,-0.27,1.28,2.17,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.20,-14.91,47.51,25.51,75232.0,-27745.0,158350.0,-2.87,53.0,H-LC,1.92,49.0,-0.37,1.11,16.69,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.24,-35.62,110.02,35.20,91134.0,-45836.0,82834.0,8.59,21.0,X-SC,12.65,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.36,-13.99,72.61,48.45,125213.0,-28060.0,172446.0,-20.09,28.0,X-MC,7.49,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
76,TMPV,600.00,-1.20,-15.19,67.64,42.18,158564.0,-41986.0,234424.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
56,QUESS,424.00,0.93,-28.65,100.19,42.84,46472.0,-18622.0,46384.0,-53.17,38.0,X-SC,37.25,83.0,-0.40,0.32,0.93,XY24,NTT,MISC
54,PAGEIND,51605.08,0.40,-4.63,32.19,26.07,50268.0,-7580.0,156160.0,-29.81,38.0,X-MC,8.85,55.0,-0.15,1.09,1.23,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.93,-28.65,100.19,42.84,46472.0,-18622.0,46384.0,-53.17,38.0,X-SC,37.25,83.0,-0.40,0.32,0.93,XY24,NTT,MISC
60,RELAXO,1176.00,-0.89,-47.57,190.51,52.31,144982.0,-69058.0,76102.0,-44.15,28.0,X-SC,3.79,91.0,-0.48,0.53,1.87,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.24,-35.62,110.02,35.20,91134.0,-45836.0,82834.0,8.59,21.0,X-SC,12.65,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.42,2.87,20.62,24.08,26928.0,3642.0,130592.0,-5.49,45.0,X-SC,4.74,89.0,0.14,0.91,23.05,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.70,-4.57,27.40,21.58,36645.0,-6399.0,133740.0,-51.86,34.0,X-MC,7.01,56.0,-0.17,0.94,18.40,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.27,-14.16,38.96,19.28,115694.0,-49001.0,296955.0,-25.24,64.0,X-LC,7.63,1.0,-0.42,2.08,9.78,X40,ATH,IT
41,INFY,2275.00,0.55,5.18,47.25,54.87,155493.0,16197.0,329085.0,-18.16,60.0,X-LC,3.31,2.0,0.10,2.30,13.89,X40,BTT,IT
76,TMPV,600.00,-1.20,-15.19,67.64,42.18,158564.0,-41986.0,234424.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.51,-10.04,50.81,35.67,144368.0,-31709.0,284133.0,-20.77,40.0,X-LC,6.06,4.0,-0.22,1.99,2.42,X40N,ATH,FMCG
43,ITC,452.00,-0.29,-0.64,11.15,10.44,22172.0,-1286.0,198852.0,-40.74,47.0,X-LC,0.76,5.0,-0.06,1.39,4.89,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.51,-14.07,111.42,81.67,89817.0,-13199.0,80611.0,7244.44,48.0,L-SC,18.53,271.0,-0.15,0.56,56.11,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.74,-12.32,59.15,39.54,52465.0,-12460.0,88699.0,-31.43,68.0,M-SC,6.83,220.0,-0.24,0.62,25.07,AR,ATH,AUTO
51,MASFIN,398.61,-1.64,-4.52,27.82,22.05,26027.0,-4425.0,93555.0,-17.80,52.0,H-SC,7.24,164.0,-0.17,0.66,35.99,XR,ATH,FINANCE
70,SURYODAY,216.00,1.17,-18.72,51.37,23.03,74771.0,-33517.0,145554.0,57.89,50.0,H-SC,11.17,167.0,-0.45,1.02,44.42,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.16,-9.13,10.07,0.02,15973.0,-15944.0,158615.0,141.63,71.0,M-SC,5.44,234.0,-1.00,1.11,37.97,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.16,-9.13,10.07,0.02,15973.0,-15944.0,158615.0,141.63,71.0,M-SC,5.44,234.0,-1.00,1.11,37.97,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.09,0.64,20.18,20.95,51132.0,1610.0,253378.0,-0.35,76.0,X-LC,12.74,31.0,0.03,1.77,36.50,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.41,-37.25,111.76,32.89,54148.0,-28756.0,48450.0,-698.49,65.0,L-MC,5.62,259.0,-0.53,0.34,33.43,XR,NTT,BANKS
50,LTIM,7201.88,1.07,9.50,20.24,31.66,54552.0,23377.0,269528.0,5.43,64.0,H-LC,12.54,43.0,0.43,1.89,49.55,X200,ATH,IT
12,BERGEPAINT,680.00,0.92,4.20,16.26,21.14,38517.0,9545.0,236884.0,-3.22,67.0,X-MC,3.65,74.0,0.25,1.66,33.52,XY24,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.79
1,25,44.67
2,50,75.43


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.50
MC    29.29
LC    25.16
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.87
X40      21.18
X40N     11.82
XR        9.65
XY25      9.14
AR        8.95
OX40N     7.68
X200      1.89
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.95
X-MC    22.87
X-LC    19.99
M-SC    12.03
X-SC     8.14
H-MC     4.70
H-LC     3.00
L-SC     1.38
M-MC     1.38
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.78,-4.95,38.89
IT,13.85,-14.23,73.20
FINANCE,9.43,-15.26,63.97
MISC,7.40,-22.70,72.97
PAINTS,6.00,-7.64,23.63
ELECTRICAL,5.89,-11.59,52.47
INSURANCE,4.41,-0.35,35.72
PHARMA,3.83,-1.95,34.58
AUTO,3.33,-20.63,72.35


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3052560.0,21
AR,1291729.0,10
XR,1284730.0,13
X40,1011909.0,14
X40N,758945.0,9
OX40N,695162.0,10
XY25,335364.0,6
SR,279155.0,2
MH,73175.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3556222.0,25
M-SC,1341644.0,15
X-MC,1215458.0,16
X-LC,906254.0,11
X-SC,755789.0,8
H-MC,406457.0,3
L-SC,262645.0,3
H-LC,129784.0,2
M-LC,119908.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218553.0      6
           AR         876438.0      5
           XR         785798.0      7
M-SC       XY24       784712.0      6
X-MC       X40        442950.0      7
X-LC       X40        390327.0      5
X-MC       XY24       347680.0      3
H-SC       OX40N      323103.0      4
X-SC       X40N       295298.0      3
M-SC       OX40N      283866.0      5
X-SC       XY24       281859.0      3
H-SC       SR         279155.0      2
X-MC       X40N       250266.0      4
X-LC       XY24       230716.0      2
H-MC       AR         217417.0      2
X-LC       X40N       213381.0      2
H-MC       XY24       189040.0      1
X-SC       X40        178632.0      2
X-MC       XY25       174562.0      2
L-SC       XR         174452.0      2
M-SC       XR         150424.0      2
           AR         122642.0      2
M-LC       XR         119908.0      1
L-SC       OX40N       88193.0      1
H-LC       AR          75232.0      1
H-SC       MH          73175.0      1
X-LC       XY25        71830.0      2
H-LC       X200        54552.0      1
L-MC       XR          54148.0      1
M-MC       XY25        48795.0      1
L-LC       XY25        40177.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
